# Testing Sargassum Data Review in EE / Python

In [1]:
# Import GEE & initialize
import ee
# Trigger the authentication flow. 
# -- I think this was a onetime thing, but keeping here in case
# ee.Authenticate()
# Initialize the library.
ee.Initialize()
import geemap

## Set up the interactive map.  Uses geemap - https://geemap.org/

In [2]:
Map = geemap.Map(center=[20,-87], zoom=8)  # Center on Quintana Roo
Map.add_basemap('Esri Ocean')
Map.add_basemap('Esri Satellite')
Map.add_basemap('HYBRID')
Map

Map(center=[20, -87], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

## Add Data Layers

In [3]:
# Data sets
classified = ee.Image("projects/ee-abnatcap/assets/sargassum/sargassum_20190507_subNoSWIR")  #_16QDH")
training = ee.FeatureCollection("projects/ee-abnatcap/assets/sargassum/trainingsites20190507")
s2image = ee.Image('COPERNICUS/S2_SR/20190507T160839_20190507T162405_T16QDH')

classified_all = ee.Image("projects/ee-abnatcap/assets/sargassum/sargassum_20190507_16QDH_allbands")  #_16QDH")
classified_best = ee.Image("projects/ee-abnatcap/assets/sargassum/sargassum_20190507_16QDH_lowcorr")  #_16QDH")
classified_subNoSWIR = ee.Image("projects/ee-abnatcap/assets/sargassum/sargassum_20190507_16QDH_subNoSWIR")  #_16QDH")
classified_subSWIR = ee.Image("projects/ee-abnatcap/assets/sargassum/sargassum_20190507_16QDH_subSWIR")  #_16QDH")


image_date = ee.Date('2019-05-07')
print('Image Date', image_date.getInfo())
QR_multipoint = ee.Geometry.MultiPoint([[-87.4, 21.2],[-86.6, 21.2],[-87.4, 20.3],[-86.6, 20.3],[-87.4, 19.3],[-87.4, 18.3]]);
S2bands = ee.List(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12','QA60'])

s2collection = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(image_date,image_date.advance(1,'day')).filterBounds(QR_multipoint).select(S2bands)
# print(s2collection.getInfo())

count = s2collection.size()
print('Count: ', count.getInfo())

s2mosaic = s2collection.mosaic()

# dem = ee.Image('USGS/SRTMGL1_003')
# landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
# landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003')
# states = ee.FeatureCollection("TIGER/2018/States")

Image Date {'type': 'Date', 'value': 1557187200000}
Count:  6


In [4]:
# Visualization parameters
sargassum_vis = {
    'min': 1, 
    'max': 1, 
    'palette':'fb04ff'
}

# Sentinel-2 bands
BLUE = 'B2' # 490 nmn, 10m
GREEN = 'B3' # 560 nm, 10m
RED = 'B4' # 665 nm, 10m
REDEDGE1 = 'B5' # 705 nm, 20m
REDEDGE2 = 'B6' # 740 nm, 20m
REDEDGE3 = 'B7' # 783 nm, 20m
NIR = 'B8' # 842 nm, 10m
NIR2 = 'B8A' # 865 nm, 20m
SWIR1 = 'B11' # 1610nm, 20m
SWIR2 = 'B12' # 2190 nm, 20m

rgbVis = {
  'min': 0,
  'max': 2500,
  'bands': [RED, GREEN, BLUE],
}
cirVis = {
  'min': 0,
  'max': 2500,
  'bands': [NIR, RED, GREEN],
}
rgbVisReflectance = {
  'min': 0.0,
  'max': 0.25,
  'bands': [RED, GREEN, BLUE],
}
cirVisReflectance = {
  'min': 0.0,
  'max': 0.25,
  'bands': [NIR, RED, GREEN],
}

# Training points and classified data
vis_params = {
    'color': '000000', 
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 0.5,
    'lineType': 'solid', 
    'fillColorOpacity': 0.66    
}

palette = ['d825ff', '3dd626', 'ffc372', '0b4a8b', 'd4d656']



In [5]:
# PLANET Images 
PlanetScope_Collection = ee.ImageCollection("projects/ee-abnatcap/assets/PSScene4Band_MAR")
PS_201905_Collection = PlanetScope_Collection.filterDate(image_date, image_date.advance(2,'day')) 

PlanetRGBvis = {"opacity":1,"bands":["B3","B2","B1"],"min":405.79,"max":4499.71,"gamma":2.331}
PlanetCIRvis = {"opacity":1,"bands":["B4","B2","B3"],"min":405.79,"max":4499.71,"gamma":2.331}

Map.addLayer(PS_201905_Collection, PlanetRGBvis, "Planet RGB", False)
Map.addLayer(PS_201905_Collection, PlanetCIRvis, "Planet CIR", False)

In [6]:
# Map.addLayer(s2image, cirVis, "S2 CIR - 2019-05-07", False)
# Map.addLayer(s2image, rgbVis, "S2 RGB - 2019-05-07", False)
# Map.addLayer(s2collection, cirVis, "S2 CIR collection - 2019-05-07", False)
Map.addLayer(s2mosaic, cirVis, "S2 CIR mosaic - 2019-05-07", False)
Map.addLayer(s2mosaic, rgbVis, "S2 RGB mosaic - 2019-05-07", False)


Map.addLayer(classified, sargassum_vis, 'Sargassum - region mosaic', True)
Map.addLayer(classified_all, {'min': 1, 'max': 1, 'palette':'42f545'},'T16QDH - all bands', True) # green
Map.addLayer(classified_best, {'min': 1, 'max': 1, 'palette':'f5d742'}, 'T16QDH - best bands', True) # gold
Map.addLayer(classified, {'min': 1, 'max': 1, 'palette':'42ecf5'}, 'T16QDH - subset w/SWIR', True) # cyan
Map.addLayer(classified, {'min': 1, 'max': 1, 'palette':'bc42f5'}, 'T16QDH - subset no SWIR', True) # purple



Map.addLayer(training, {}, 'training', False)

In [7]:
Map.add_styled_vector(training, column="lc_code", palette=palette, layer_name="land cover", **vis_params)

In [8]:
classified_stats = geemap.image_stats(classified)
classified_stats.getInfo()

{'max': {'classification': 0},
 'mean': {'classification': 0},
 'min': {'classification': 0},
 'std': {'classification': 0},
 'sum': {'classification': 0}}